To finetune the Resnet model, we first generate adversarial training images and adversarial validation images by the following commands:
```
python launch_resnet_attack.py --batch_num 20 --batch_size 100 --results 'adv_train_images'
python launch_resnet_attack.py --batch_num 20 --batch_size 100 --results 'adv_val_images' --seed 0
```
The params are all default.
Then we use `adv_train_images` to train the Resnet model and save it.


In [1]:
import warnings
warnings.filterwarnings("ignore")
!python launch_resnet_attack.py --batch_num 20 --batch_size 100 --results 'adv_train_images'
!python launch_resnet_attack.py --batch_num 20 --batch_size 100 --results 'adv_val_images' --seed 0

Loading model...
Loading data...
===Launching PGD attack on 20 batches of data===
Attack configs: eps = 0.03137254901960784, alpha = 0.00784313725490196, steps = 20, batch size = 100
100%|███████████████████████████████████████████| 20/20 [00:49<00:00,  2.49s/it]
Accuracy on original images: 92.7%
Accuracy on adversarial images: 2.0%
Loading model...
Loading data...
===Launching PGD attack on 20 batches of data===
Attack configs: eps = 0.03137254901960784, alpha = 0.00784313725490196, steps = 20, batch size = 100
100%|███████████████████████████████████████████| 20/20 [00:49<00:00,  2.47s/it]
Accuracy on original images: 91.5%
Accuracy on adversarial images: 1.9%


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader, TensorDataset

ADV_TRAIN_PATH = './results/adv_train_images'
BATCH_SIZE = 100
BATCH_NUM = 20
torch.manual_seed(1234)

checkpoint = torch.load(ADV_TRAIN_PATH)
adv_images = checkpoint['adv_images']
labels = checkpoint['labels']

dataset = TensorDataset(adv_images, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

torch.save(model.state_dict(), 'results/resnet50_finetuned.pth')

Epoch [1/10], Loss: 4.3008
Epoch [2/10], Loss: 1.4592
Epoch [3/10], Loss: 0.8949
Epoch [4/10], Loss: 0.6399
Epoch [5/10], Loss: 0.4656
Epoch [6/10], Loss: 0.3578
Epoch [7/10], Loss: 0.2765
Epoch [8/10], Loss: 0.2294
Epoch [9/10], Loss: 0.1843
Epoch [10/10], Loss: 0.1558


We load the finetuned model and use validation images to test it.

The accuracy on adversarial validation images reaches 72.90%, much higher than the model without finetuned (1.9%).

In [3]:
model = resnet50(pretrained=False)
model.load_state_dict(torch.load('results/resnet50_finetuned.pth'))
model = model.to(device)

ADV_VAL_PATH = './results/adv_val_images'
BATCH_SIZE = 100
BATCH_NUM = 20

checkpoint = torch.load(ADV_VAL_PATH)
val_images = checkpoint['adv_images']
labels = checkpoint['labels']

dataset = TensorDataset(val_images, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

model.eval()
correct = 0
total = 0

for inputs, labels in dataloader:
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = model(inputs).softmax(1)
    predictions = outputs.argmax(dim=1)
  correct += torch.sum(predictions == labels).item()
  total += len(labels)


acc = correct / total
print(f"Accuracy on adversarial validation images: {acc * 100}%")

Accuracy on adversarial validation images: 72.89999999999999%
